# Business Understanding

# Data Understanding

About this movie dataset. IT's from IMDB and Box Office Mojo

## Data Preparation 

# Previewing IMDB data

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('im.db')

q = """
SELECT name
FROM sqlite_master
WHERE type = 'table';"""

tables = pd.read_sql(q, conn)
print(tables)


            name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


In [2]:
for table_name in tables['name']:
    print(f"\nTable: {table_name}")
    df = pd.read_sql(f"SELECT * FROM {table_name} LIMIT 5;", conn)
    print(df)



Table: movie_basics
    movie_id                    primary_title              original_title  \
0  tt0063540                        Sunghursh                   Sunghursh   
1  tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
2  tt0069049       The Other Side of the Wind  The Other Side of the Wind   
3  tt0069204                  Sabse Bada Sukh             Sabse Bada Sukh   
4  tt0100275         The Wandering Soap Opera       La Telenovela Errante   

   start_year  runtime_minutes                genres  
0        2013            175.0    Action,Crime,Drama  
1        2019            114.0       Biography,Drama  
2        2018            122.0                 Drama  
3        2018              NaN          Comedy,Drama  
4        2017             80.0  Comedy,Drama,Fantasy  

Table: directors
    movie_id  person_id
0  tt0285252  nm0899854
1  tt0462036  nm1940585
2  tt0835418  nm0151540
3  tt0835418  nm0151540
4  tt0878654  nm0089502

Table: known_for
   per

## don't want: foreign language as main language, movies not in date range, 

## IMDB Query: data from 2010-2018; 
## Questions: what is persons and principles tables, how are they different? 


## Basics table: 
### filter out: outside date range, missing genre or runtime, 

## akas
### what is type in akas-what does each category mean? what attributes do we want? 
### only US region

## persons, principles
### filter died before 2010

In [ ]:
#SELECT DISTINCT your_column
#FROM your_table;

# Previewing Box Office Mojo Data

In [3]:
df = pd.read_csv('bom.movie_gross.csv.gz', compression = 'gzip')
df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None


In [5]:
print(df.describe(include = 'all'))

            title studio  domestic_gross foreign_gross         year
count        3387   3382    3.359000e+03          2037  3387.000000
unique       3386    257             NaN          1204          NaN
top     Bluebeard    IFC             NaN       1200000          NaN
freq            2    166             NaN            23          NaN
mean          NaN    NaN    2.874585e+07           NaN  2013.958075
std           NaN    NaN    6.698250e+07           NaN     2.478141
min           NaN    NaN    1.000000e+02           NaN  2010.000000
25%           NaN    NaN    1.200000e+05           NaN  2012.000000
50%           NaN    NaN    1.400000e+06           NaN  2014.000000
75%           NaN    NaN    2.790000e+07           NaN  2016.000000
max           NaN    NaN    9.367000e+08           NaN  2018.000000


In [6]:
df['title'].value_counts()

title
Bluebeard               2
Before We Go            1
Knock Knock             1
Kindergarten Teacher    1
Welcome to Leith        1
                       ..
Monsters University     1
Gravity                 1
Man of Steel            1
Thor: The Dark World    1
An Actor Prepares       1
Name: count, Length: 3386, dtype: int64

In [7]:
df['studio'].value_counts()

studio
IFC           166
Uni.          147
WB            140
Fox           136
Magn.         136
             ... 
E1              1
PI              1
ELS             1
PalT            1
Synergetic      1
Name: count, Length: 257, dtype: int64

In [25]:
df['year'].value_counts()

year
2010    308
2011    290
2012    243
2014    234
2013    200
2016    192
2015    190
2017    177
2018    173
Name: count, dtype: int64

In [9]:
df.isna().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

# Data Cleaning


# Box Office Mojo

### removing rows with null values- only interested in rows with financial info that can come back to studios

In [10]:
df=df.dropna()

In [11]:
df.isna().sum()

title             0
studio            0
domestic_gross    0
foreign_gross     0
year              0
dtype: int64

### Changing title and studio columns to string data types for easier handling- *don't need to do this- record in data cleaning notes

### removing any white spaces

In [27]:
df['title'] = df['title'].str.strip()
df['studio'] = df['studio'].str.strip()
df['year'] = pd.to_datetime(df['year'])

### get a look at studio names for typos, etc

In [24]:
print(sorted(df['studio'].unique().tolist()))

['3D', 'A24', 'AF', 'AGF', 'AR', 'ATO', 'Abr.', 'Affirm', 'Amazon', 'Ampl.', 'Anch.', 'Annapurna', 'App.', 'ArcEnt', 'Argo.', 'Arth.', 'Aviron', 'BBC', 'BG', 'BGP', 'BH Tilt', 'BST', 'BV', 'Blue Fox', 'CBS', 'CE', 'CGld', 'CJ', 'CL', 'CLS', 'CP', 'CZ', 'Cohen', 'Crnth', 'DR', 'Da.', 'Darin Southa', 'Distrib.', 'Drft.', 'E1', 'EC', 'ENTMP', 'EOne', 'Elev.', 'EpicPics', 'Eros', 'FCW', 'FD', 'FIP', 'FInd.', 'FM', 'FOAK', 'FR', 'FRun', 'FUN', 'First', 'Focus', 'Fox', 'FoxS', 'Free', 'GK', 'Global Road', 'Gold.', 'Good Deed', 'Greenwich', 'Grindstone', 'GrtIndia', 'HC', 'HTR', 'Hann.', 'ICir', 'IFC', 'IM', 'ITL', 'IVP', 'IW', 'Icar.', 'Imag.', 'Imax', 'JBG', 'JS', 'Jan.', 'KC', 'KE', 'KKM', 'KL', 'Kino', 'LD', 'LG/S', 'LGF', 'LGP', 'Libre', 'Lorb.', 'MBox', 'MGM', 'MNE', 'MOM', 'MPFT', 'MPI', 'Magn.', 'Mira.', 'Mont.', 'NAV', 'NFC', 'NGE', 'NM', 'NYer', 'NeoC', 'Neon', 'OMNI/FSR', 'ORF', 'Olive', 'Orch.', 'Osci.', 'Outs', 'Over.', 'P/DW', 'P4', 'PBS', 'PDA', 'PFR', 'PH', 'PM&E', 'PNT', 'Pal

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2007 entries, 0 to 3353
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           2007 non-null   object        
 1   studio          2007 non-null   object        
 2   domestic_gross  2007 non-null   float64       
 3   foreign_gross   2007 non-null   object        
 4   year            2007 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 94.1+ KB


### filtering for smaller studios


In [14]:
#filtered_df = df[df['studio'].groupby(df['studio']).transform('count').between(1, 20)]


### removes low value studio rows- smaller studios

In [15]:
#df = df[df.groupby('studio')['studio'].transform('count')>= 3]
#df['studio'].value_counts()

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2007 entries, 0 to 3353
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           2007 non-null   object 
 1   studio          2007 non-null   object 
 2   domestic_gross  2007 non-null   float64
 3   foreign_gross   2007 non-null   object 
 4   year            2007 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 94.1+ KB


# Exploratory Data Analysis

## Filtering ideas: at what point does profitability happen, how many movies? 

## Both datasets: How does actor, director, crew quality impact this?

## Did director, studio go on to make more movies? btw 2010 and 2018


# Conclusions

## Limitations

# no info on: profits, pay for crew or actors, other revenue streams, cost to produce the film; might be better to make a smaller film with less risk.

## Recommendations

# Next Steps